In [1]:
with open("/mnt/d/forCoding_code/_init_tools/x__params.py", "r") as f:
    txt = f.read()
exec(txt)

with open("/mnt/d/forCoding_code/_init_tools/report_macro_0516.py", "r") as f:
    txt = f.read()
exec(txt)

with open("/mnt/d/forCoding_code/_init_tools/x__tool_here1.py", "r") as f:
    exec(f.read())
with open("/mnt/d/forCoding_code/_init_tools/x__tool_here2.py", "r") as f:
    exec(f.read())

wasted_dir = "./wasted"
if not os.path.exists(wasted_dir):
    os.makedirs(wasted_dir)

pd.set_option('display.max_rows',200)
pd.set_option('display.max_columns',200)

storage dir: /mnt/d/forCoding_data/Tianchi_Ecommerce6/plan_1
code dir: /mnt/d/forCoding_code/Tianchi_Ecommerce6/plan_1


In [39]:
!sh run.sh

Starting Prediction...
/home/xiuminke/miniconda3/envs/ml12/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
Running Extraction...
Some weights of ExtractionModel were not initialized from the model checkpoint at /mnt/d/ModelScopeModels/google-bert/bert-base-chinese/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|█████████████████████████████████████████| 140/140 [00:13<00:00, 10.54it/s]
Running Classification...
Some weights of ClassificationModel were not initialized from the model checkpoint at /mnt/d/ModelScopeModels/google-bert/bert-base-chinese/ and are newly initialized: ['cat_classifier.bias'

In [1]:
import os
import sys
import torch
from transformers import BertTokenizerFast as BertTokenizer
from config import Config
from utils_loc import load_train_data, ExtractionDataset

def show_sample(dataset, idx=0, limit=30):
    item, text, rid, offset_mapping = dataset[idx]
    input_ids = item["input_ids"]
    attention_mask = item["attention_mask"]
    token_type_ids = item["token_type_ids"]
    labels = item.get("labels")

    print(f"id: {rid}")
    print(f"text: {text}")
    print(f"input_ids.shape: {tuple(input_ids.shape)}")
    print(f"attention_mask.sum: {int(attention_mask.sum().item())}")
    if labels is not None:
        print(f"labels.shape: {tuple(labels.shape)}")

    tokenizer = BertTokenizer.from_pretrained(Config.BERT_PATH)
    tokens = tokenizer.convert_ids_to_tokens(input_ids.tolist())

    cnt = 0
    for i in range(len(tokens)):
        if attention_mask[i].item() != 1:
            continue
        start, end = offset_mapping[i].tolist()
        if start == 0 and end == 0:
            continue
        tok = tokens[i]
        tag = None
        if labels is not None:
            tag_idx = int(labels[i].item())
            tag = Config.IDX2TAG.get(tag_idx, str(tag_idx))
        line = f"{i:03d}  token={tok}  span=({start},{end})  text='{text[start:end]}'"
        if tag is not None:
            line += f"  tag={tag}"
        print(line)
        cnt += 1
        if cnt >= limit:
            break

# def main():
idx = int(os.environ.get("IDX", "0"))
limit = int(os.environ.get("LIMIT", "30"))

print("loading tokenizer and dataset ...")
tokenizer = BertTokenizer.from_pretrained(Config.BERT_PATH)
reviews, labels = load_train_data()
dataset = ExtractionDataset(reviews, labels, tokenizer, Config.MAX_LEN)
print(f"dataset size: {len(dataset)}")

show_sample(dataset, idx=idx, limit=limit)

/home/xiuminke/miniconda3/envs/ml12/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/home/xiuminke/miniconda3/envs/ml12/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading tokenizer and dataset ...
dataset size: 3229
id: 1
text: 很好，超值，很好用
input_ids.shape: (512,)
attention_mask.sum: 11
labels.shape: (512,)
001  token=很  span=(0,1)  text='很'  tag=B-OP
002  token=好  span=(1,2)  text='好'  tag=I-OP
003  token=，  span=(2,3)  text='，'  tag=O
004  token=超  span=(3,4)  text='超'  tag=B-OP
005  token=值  span=(4,5)  text='值'  tag=I-OP
006  token=，  span=(5,6)  text='，'  tag=O
007  token=很  span=(6,7)  text='很'  tag=B-OP
008  token=好  span=(7,8)  text='好'  tag=I-OP
009  token=用  span=(8,9)  text='用'  tag=I-OP


In [4]:
reviews

,id,Reviews
0,1,很好，超值，很好用
1,2,很好，遮暇功能差一些，总体还不错
2,3,包装太随便了，连个包装盒都没有，第一感觉很不好
3,4,宝贝收到了，产品非常的不好，简直就是个垃圾，我都扔了。
4,5,活动价很是划算，买一送一共60片才花八十五块，天天用都不心疼啊
...,...,...
3224,3225,***，大品牌值得信赖，皮肤亮白！服务周到！
3225,3226,之前也有买过，这是并不是第一次购买，应该是之前一样吧，价格挺优惠的
3226,3227,物流很快，包装不错，第一次使用这个牌子，期待&hellip
3227,3228,感觉还可以，唯一的遗憾就是面膜放了香精，希望***官方能把香精去掉就完美了


In [9]:
labels[labels.id==14]

,id,AspectTerms,A_start,A_end,OpinionTerms,O_start,O_end,Categories,Polarities
27,14,_,,,很好的,0,3,整体,正面
28,14,_,,,不错,28,30,整体,正面


In [10]:
labels[labels.AspectTerms!="_"]

,id,AspectTerms,A_start,A_end,OpinionTerms,O_start,O_end,Categories,Polarities
4,2,遮暇功能,3,7,差一些,7,10,功效,负面
6,3,包装,0,2,太随便了,2,6,包装,负面
7,3,包装盒,9,12,没有,13,15,包装,负面
12,6,香味,14,16,淡淡的,11,14,气味,正面
26,13,物流,15,17,快,18,19,物流,正面
...,...,...,...,...,...,...,...,...,...
6624,3227,物流,0,2,很快,2,4,物流,正面
6625,3227,包装,5,7,不错,7,9,包装,正面
6626,3228,香精,17,19,_,,,成分,负面
6629,3229,补水,15,17,棒棒的,17,20,功效,正面


In [11]:
labels.Categories.value_counts()

Categories
整体      2822
使用体验    1042
功效       726
价格       696
物流       517
气味       225
包装       195
真伪       161
服务        86
其他        65
成分        61
尺寸        24
新鲜度       13
Name: count, dtype: int64

In [ ]:
({'input_ids': tensor([ 101, 2523, 1962, 8024, 6631,  966, 8024, 2523, 1962, 4500,  102,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
、]),
  'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
]),
  'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
]),
  'labels': tensor([0, 3, 4, 0, 3, 4, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
])},
 '很好，超值，很好用',
 1,
 tensor([[0, 0],
         [0, 1],
         [1, 2],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]]))

In [13]:
dataset[1]

({'input_ids': tensor([ 101, 2523, 1962, 8024, 6902, 3259, 1216, 5543, 2345,  671,  763, 8024,
          2600,  860, 6820,  679, 7231,  102,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
   